# Logistic Regression Consulting Project

## Binary Customer Churn

A marketing agency has many customers that use their service to produce ads for the client/customer websites. They've noticed that they have quite a bit of churn in clients. They basically randomly assign account managers right now, but want you to create a machine learning model that will help predict which customers will churn (stop buying their service) so that they can correctly assign the customers most at risk to churn an account manager. Luckily they have some historical data, can you help them out? Create a classification algorithm that will help classify whether or not a customer churned. Then the company can test this against incoming data for future customers to predict which customers will churn and assign them an account manager.

The data is saved as customer_churn.csv. Here are the fields and their definitions:

    Name : Name of the latest contact at Company
    Age: Customer Age
    Total_Purchase: Total Ads Purchased
    Account_Manager: Binary 0=No manager, 1= Account manager assigned
    Years: Totaly Years as a customer
    Num_sites: Number of websites that use the service.
    Onboard_date: Date that the name of the latest contact was onboarded
    Location: Client HQ Address
    Company: Name of Client Company
    
Once you've created the model and evaluated it, test out the model on some new data (you can think of this almost like a hold-out set) that your client has provided, saved under new_customers.csv. The client wants to know which customers are most likely to churn given this data (they don't have the label yet).

**Imports**

In [36]:
import findspark
findspark.init('/home/sedat/spark-3.3.2-bin-hadoop3')
from pyspark.sql import SparkSession
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator
from pyspark.sql.functions import count, col, isnan, when
from pyspark.ml.feature import StringIndexer, OneHotEncoder, VectorIndexer, VectorAssembler
from pyspark.ml import Pipeline
from pyspark.sql.functions import (dayofmonth, dayofyear, 
                                   weekofyear, format_number,
                                   hour, month, year, date_format, days)

**Start session and read csv file**

In [4]:
spark = SparkSession.builder.appName('customer_churn').getOrCreate()

In [59]:
df = spark.read.csv('customer_churn.csv', inferSchema=True, header=True) 

In [60]:
df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)



**Total observation units**

In [61]:
df.count()

900

**First row**

In [62]:
df.head(1)[0]

Row(Names='Cameron Williams', Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Onboard_date=datetime.datetime(2013, 8, 30, 7, 0, 40), Location='10265 Elizabeth Mission Barkerburgh, AK 89518', Company='Harvey LLC', Churn=1)

**Select columns that working on**

In [63]:
df.columns

['Names',
 'Age',
 'Total_Purchase',
 'Account_Manager',
 'Years',
 'Num_Sites',
 'Onboard_date',
 'Location',
 'Company',
 'Churn']

**Parse onboard_date and add data as column**

In [64]:
new_df = df.withColumns({
    'Onboard_year': year(df['Onboard_date']),
    'Onboard_month': month(df['Onboard_date']),
    'Onboard_date_': dayofmonth(df['Onboard_date']),
    'Onboard_hour': hour(df['Onboard_date'])
})
new_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Onboard_year: integer (nullable = true)
 |-- Onboard_month: integer (nullable = true)
 |-- Onboard_date_: integer (nullable = true)
 |-- Onboard_hour: integer (nullable = true)



**Select columns**

In [66]:
my_col = new_df.select(['Age',
                    'Total_Purchase',
                    'Account_Manager',
                    'Years',
                    'Num_Sites',
                    'Churn',
                    'Onboard_year',
                    'Onboard_month',
                    'Onboard_date_',
                    'Onboard_hour'])

In [67]:
my_col.show()

+----+--------------+---------------+-----+---------+-----+------------+-------------+-------------+------------+
| Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|Onboard_year|Onboard_month|Onboard_date_|Onboard_hour|
+----+--------------+---------------+-----+---------+-----+------------+-------------+-------------+------------+
|42.0|       11066.8|              0| 7.22|      8.0|    1|        2013|            8|           30|           7|
|41.0|      11916.22|              0|  6.5|     11.0|    1|        2013|            8|           13|           0|
|38.0|      12884.75|              0| 6.67|     12.0|    1|        2016|            6|           29|           6|
|42.0|       8010.76|              0| 6.71|     10.0|    1|        2014|            4|           22|          12|
|37.0|       9191.58|              0| 5.56|      9.0|    1|        2016|            1|           19|          15|
|48.0|      10356.02|              0| 5.12|      8.0|    1|        2009|            3|  

**Check missing value and delete them**

In [68]:
my_col.select([count(when(isnan(c) | col(c).isNull(), c)).alias(c) for c in my_col.columns]).show()

+---+--------------+---------------+-----+---------+-----+------------+-------------+-------------+------------+
|Age|Total_Purchase|Account_Manager|Years|Num_Sites|Churn|Onboard_year|Onboard_month|Onboard_date_|Onboard_hour|
+---+--------------+---------------+-----+---------+-----+------------+-------------+-------------+------------+
|  0|             0|              0|    0|        0|    0|           0|            0|            0|           0|
+---+--------------+---------------+-----+---------+-----+------------+-------------+-------------+------------+



There is no missing value in our data. That's good!

**Assembler and final data**

In [84]:
assembler = VectorAssembler(inputCols=['Age',
                                        'Total_Purchase',
                                        'Account_Manager',
                                        'Years',
                                        'Num_Sites',
                                        'Onboard_year',
                                        'Onboard_month',
                                        'Onboard_date_',
                                        'Onboard_hour'], outputCol='features')

In [85]:
output = assembler.transform(my_col)

In [86]:
output.head(1)

[Row(Age=42.0, Total_Purchase=11066.8, Account_Manager=0, Years=7.22, Num_Sites=8.0, Churn=1, Onboard_year=2013, Onboard_month=8, Onboard_date_=30, Onboard_hour=7, features=DenseVector([42.0, 11066.8, 0.0, 7.22, 8.0, 2013.0, 8.0, 30.0, 7.0]))]

In [87]:
final_data = output.select('Churn', 'features')

In [88]:
final_data.show()

+-----+--------------------+
|Churn|            features|
+-----+--------------------+
|    1|[42.0,11066.8,0.0...|
|    1|[41.0,11916.22,0....|
|    1|[38.0,12884.75,0....|
|    1|[42.0,8010.76,0.0...|
|    1|[37.0,9191.58,0.0...|
|    1|[48.0,10356.02,0....|
|    1|[44.0,11331.58,1....|
|    1|[32.0,9885.12,1.0...|
|    1|[43.0,14062.6,1.0...|
|    1|[40.0,8066.94,1.0...|
|    1|[30.0,11575.37,1....|
|    1|[45.0,8771.02,1.0...|
|    1|[45.0,8988.67,1.0...|
|    1|[40.0,8283.32,1.0...|
|    1|[41.0,6569.87,1.0...|
|    1|[38.0,10494.82,1....|
|    1|[45.0,8213.41,1.0...|
|    1|[43.0,11226.88,0....|
|    1|[53.0,5515.09,0.0...|
|    1|[46.0,8046.4,1.0,...|
+-----+--------------------+
only showing top 20 rows



**Split data to train and test**

In [89]:
train_data, test_data = final_data.randomSplit([0.7, 0.3])

**Model**

In [90]:
log_model = LogisticRegression(labelCol='Churn')

In [92]:
logModel = log_model.fit(train_data)

In [98]:
train_summary = logModel.summary
train_summary.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|[25.0,9672.03,0.0...|[4.99159546311291...|[0.99325104295144...|       0.0|
|  0.0|[26.0,8939.61,0.0...|[6.12686233006000...|[0.99782133586242...|       0.0|
|  0.0|[27.0,8628.8,1.0,...|[5.71512583390827...|[0.99671509041381...|       0.0|
|  0.0|[28.0,8670.98,0.0...|[7.60658937289954...|[0.99950308243656...|       0.0|
|  0.0|[28.0,9090.43,1.0...|[1.63477601229130...|[0.83682285456359...|       0.0|
|  0.0|[28.0,11245.38,0....|[3.45196315406739...|[0.96928963219209...|       0.0|
|  0.0|[29.0,5900.78,1.0...|[3.85050948608983...|[0.97917404757675...|       0.0|
|  0.0|[29.0,9378.24,0.0...|[4.71679690272487...|[0.99113550161241...|       0.0|
|  0.0|[29.0,9617.59,0.0...|[4.67722589906544...|[0.99078099040813...|       0.0|
|  0.0|[29.0,112

In [100]:
train_summary.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                613|                613|
|   mean|0.16639477977161501|0.12234910277324633|
| stddev| 0.3727387870498555| 0.3279561815442461|
|    min|                0.0|                0.0|
|    max|                1.0|                1.0|
+-------+-------------------+-------------------+



**Model Evaluation**

In [102]:
my_eval = BinaryClassificationEvaluator(rawPredictionCol='predictions', labelCol='Churn')

In [105]:
predictions_and_labels = logModel.evaluate(test_data)

In [107]:
predictions_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|Churn|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|    0|[22.0,11254.38,1....|[4.61578065526162...|[0.99020248485289...|       0.0|
|    0|[28.0,11128.95,1....|[4.14919280362649...|[0.98446790552965...|       0.0|
|    0|[28.0,11204.23,0....|[1.59750311826551...|[0.83166912211339...|       0.0|
|    0|[29.0,10203.18,1....|[4.02739875045701...|[0.98249138879668...|       0.0|
|    0|[29.0,13255.05,1....|[4.40851142336361...|[0.98797312091721...|       0.0|
|    0|[30.0,8677.28,1.0...|[3.63984631637689...|[0.97441538062743...|       0.0|
|    0|[30.0,12788.37,0....|[2.26606107855388...|[0.90602695560676...|       0.0|
|    0|[30.0,13473.35,0....|[2.71940137546835...|[0.93816181410614...|       0.0|
|    0|[31.0,7073.61,0.0...|[2.75304048290439...|[0.94008483537989...|       0.0|
|    0|[31.0,882

In [108]:
predictions_and_labels.predictions.describe().show()

+-------+-------------------+-------------------+
|summary|              Churn|         prediction|
+-------+-------------------+-------------------+
|  count|                287|                287|
|   mean| 0.1672473867595819|0.13588850174216027|
| stddev|0.37384846709334274|0.34326867920925896|
|    min|                  0|                0.0|
|    max|                  1|                1.0|
+-------+-------------------+-------------------+



**Roc auc**

In [110]:
churn_eval = BinaryClassificationEvaluator(rawPredictionCol='prediction', labelCol='Churn')

In [111]:
auc = churn_eval.evaluate(predictions_and_labels.predictions)

In [113]:
auc

0.7561453974895397

**Predict on brand new unlabeled data**

In [174]:
new_customer = spark.read.csv('new_customers.csv', inferSchema=True, header=True)

In [175]:
new_customer.count()

6

In [176]:
new_customer.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)



In [177]:
for customer in new_customer.head(6):
    print(customer)
    print('\n')

Row(Names='Andrew Mccall', Age=37.0, Total_Purchase=9935.53, Account_Manager=1, Years=7.71, Num_Sites=8.0, Onboard_date=datetime.datetime(2011, 8, 29, 18, 37, 54), Location='38612 Johnny Stravenue Nataliebury, WI 15717-8316', Company='King Ltd')


Row(Names='Michele Wright', Age=23.0, Total_Purchase=7526.94, Account_Manager=1, Years=9.28, Num_Sites=15.0, Onboard_date=datetime.datetime(2013, 7, 22, 18, 19, 54), Location='21083 Nicole Junction Suite 332, Youngport, ME 23686-4381', Company='Cannon-Benson')


Row(Names='Jeremy Chang', Age=65.0, Total_Purchase=100.0, Account_Manager=1, Years=1.0, Num_Sites=15.0, Onboard_date=datetime.datetime(2006, 12, 11, 7, 48, 13), Location='085 Austin Views Lake Julialand, WY 63726-4298', Company='Barron-Robertson')


Row(Names='Megan Ferguson', Age=32.0, Total_Purchase=6487.5, Account_Manager=0, Years=9.4, Num_Sites=14.0, Onboard_date=datetime.datetime(2016, 10, 28, 5, 32, 13), Location='922 Wright Branch North Cynthialand, NC 64721', Company='Sexton-G

In [180]:
new_customer = new_customer.withColumns({
    'Onboard_year': year(new_customer['Onboard_date']),
    'Onboard_month': month(new_customer['Onboard_date']),
    'Onboard_date_': dayofmonth(new_customer['Onboard_date']),
    'Onboard_hour': hour(new_customer['Onboard_date'])
})
new_df.printSchema()

root
 |-- Names: string (nullable = true)
 |-- Age: double (nullable = true)
 |-- Total_Purchase: double (nullable = true)
 |-- Account_Manager: integer (nullable = true)
 |-- Years: double (nullable = true)
 |-- Num_Sites: double (nullable = true)
 |-- Onboard_date: timestamp (nullable = true)
 |-- Location: string (nullable = true)
 |-- Company: string (nullable = true)
 |-- Churn: integer (nullable = true)
 |-- Onboard_year: integer (nullable = true)
 |-- Onboard_month: integer (nullable = true)
 |-- Onboard_date_: integer (nullable = true)
 |-- Onboard_hour: integer (nullable = true)



In [182]:
test_new_customers = assembler.transform(new_customer)

In [183]:
final_results = logModel.transform(test_new_customers)

In [185]:
final_results.select('Company','prediction').show()

+----------------+----------+
|         Company|prediction|
+----------------+----------+
|        King Ltd|       0.0|
|   Cannon-Benson|       1.0|
|Barron-Robertson|       1.0|
|   Sexton-Golden|       1.0|
|        Wood LLC|       0.0|
|   Parks-Robbins|       1.0|
+----------------+----------+

